In [0]:
import tensorflow as tf 
import numpy as np 
import os 
import time

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
## load cifar10 dataset 
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## reshape (None, 1) -> (None)
y_train, y_test = [y.reshape([-1]) for y in [y_train, y_test]]

## normalization 
x_train, x_test = [x /255. for x in [x_train, x_test]]

## N classㅡ
n_classes = 10
print('image shape : {}, label shape : {} '.format(x_train.shape, y_train.shape))
print('image shape : {}, label shape : {} '.format(x_test.shape, y_test.shape))
print('train minimun : {}, train_maximum : {} '.format(x_train.min(), x_train.max()))
print('tests minimun : {}, test_maximum : {} '.format(x_test.min(), x_test.max()))

Using TensorFlow backend.


image shape : (50000, 32, 32, 3), label shape : (50000,) 
image shape : (10000, 32, 32, 3), label shape : (10000,) 
train minimun : 0.0, train_maximum : 1.0 
tests minimun : 0.0, test_maximum : 1.0 


In [0]:
def image_augmentation(image, is_training, crop_h, crop_w):

    def _aug_with_train(input_x, crop_height, crop_width):
        img_h, img_w, ch = list(map(int, input_x.get_shape()[:]))

        pad_w = int(img_h * 0.2)
        pad_h = int(img_w * 0.2)

        input_x = tf.image.resize_image_with_crop_or_pad(input_x, img_h+pad_h, img_w+pad_w)
        input_x = tf.random_crop(input_x, [crop_height, crop_width, ch])
        input_x = tf.image.random_flip_left_right(input_x)
        input_x = tf.image.random_flip_up_down(input_x)

        input_x = tf.image.random_contrast(input_x, lower=0.2,upper=2.0)
        input_x = tf.image.random_brightness(input_x, max_delta=63. / 255.)
        input_x = tf.image.random_saturation(input_x, lower=0.5, upper=1.8)
        input_x = tf.image.per_image_standardization(input_x)
        return input_x

    def _aug_with_test(input_x, crop_height, crop_width):

        input_x = tf.image.resize_image_with_crop_or_pad(input_x, crop_height, crop_width)
        input_x = tf.image.per_image_standardization(input_x)
        return input_x

    image = tf.cond(is_training,
                    lambda: _aug_with_train(image, crop_h, crop_w),
                    lambda: _aug_with_test(image, crop_h, crop_w))
    return image



def images_augmentation(images, phase_train):
    with tf.name_scope('augmentation'):
        crop_h, crop_w = list(map(int, images.get_shape()[1:3]))
        images = tf.map_fn(lambda image : image_augmentation(image, phase_train, crop_h, crop_w),
                  images)
        return images

In [0]:
import numpy as np
import random

class DataProvider(object):
    def __init__(self, images, labels):
        self.n_sample = len(labels)
        self.queue = list(range(self.n_sample))
        random.shuffle(self.queue)

        self.images = images
        self.labels = labels
        self.epoch_count = 0

    def next_batch(self, batch_size):
        if len(self.queue) < batch_size:
            self.queue = list(range(self.n_sample))
            self.epoch_count += 1
        target_indices = self.queue[:batch_size]
        del self.queue[:batch_size]
        return self.images[target_indices], self.labels[target_indices]

In [0]:
# convolution helper function
def conv(input_xs ,units, k, s, padding, activation, name):
    with tf.variable_scope(name):
        ch = int(input_xs.get_shape()[-1]) # x shape : [batch, w, h, ch]

        ws = tf.get_variable('ws', [k, k, ch, units], tf.float32,
                             initializer=tf.initializers.he_normal())
        bias = tf.get_variable('bias', initializer=tf.zeros(shape=[units],
                                                          dtype=tf.float32))
        tf.add_to_collection(value=ws, name='ws')
        layer = tf.nn.conv2d(input_xs, ws, [1,s,s,1],
                             padding=padding, name='conv') + bias
        if not activation is None:
            layer = activation(layer)
    return layer

In [0]:
def batch_norm_(x, n_out, phase_train, scope='bn'):
    """
    Batch normalization on convolutional maps.
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope(scope):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        if len(x.get_shape()) == 4:
            batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        elif len(x.get_shape()) == 2:
            batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
            
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean),
                                     ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [0]:
def residual_block(input_xs, n_skip, filters, phase_train, block_name):
    """
    residual network 의 residual block을 구현합니다.
    Args:
        input_xs:    tensor, 4D dimension BHWC
        n_skip:      interger, 몇 개의 block 을 건너뛸지. 
        phase_train: boolean tf.Variable, true indicates training phase
    Return:
        layer:      Tensor, 4D Dimension BHWC
    """
    with tf.name_scope(block_name):
        layer = input_xs
        for index in range(n_skip):
            layer_name='layer_{}'.format(index)
            with tf.name_scope(layer_name):
                layer = tf.layers.Conv2D(filters, 3, 1, 'same', activation=None,
                                         use_bias=False)(layer)
                layer = batch_norm_(layer, filters, phase_train)
                
                layer = tf.nn.relu(layer)

        # Projection Layer 
        if input_xs.shape[-1] != filters:
                input_xs = tf.layers.Conv2D(filters, 1, 1, 'same', 
                                         activation=tf.nn.relu)(input_xs)
                
        return layer + input_xs

# Model 

In [9]:
# define input placeholder 
xs = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name='xs')
ys = tf.placeholder(shape=[None], dtype=tf.int32, name='ys')
lr = tf.placeholder(shape=[], dtype=tf.float32, name='lr')
phase_train = tf.placeholder(shape=[], dtype=tf.bool, name='phase_train')

xs_aug = images_augmentation(xs, phase_train)

# Stem
layer = tf.layers.Conv2D(16, 3, 1, 'same', activation=tf.nn.relu, name='stem')

# Residual 
layer = residual_block(xs , 2, 16, phase_train, 'block_1')
layer = tf.layers.MaxPooling2D(2, 2)(layer)
layer = residual_block(layer , 2, 32, phase_train, 'block_2')
layer = tf.layers.MaxPooling2D(2, 2)(layer)
layer = residual_block(layer , 2, 64, phase_train, 'block_3')
layer = tf.layers.MaxPooling2D(2, 2)(layer)
layer = residual_block(layer , 2, 128, phase_train, 'block_4')
layer = tf.layers.AveragePooling2D(4, 1)(layer)
layer = tf.layers.Flatten()(layer)

logits = tf.layers.Dense(n_classes)(layer)
print(logits)

W0705 11:40:41.744477 139837073938304 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/image_ops_impl.py:1514: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0705 11:40:41.799171 139837073938304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Tensor("dense/BiasAdd:0", shape=(?, 10), dtype=float32)


# Loss Function

In [0]:
l2_reg = tf.reduce_sum([tf.reduce_sum(var**2, axis=None) 
                        for var in tf.get_collection('ws')])
l2_beta = 0.0005
#loss 
# L2 reularization 
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=ys, logits=logits)
loss = tf.reduce_mean(loss, name='loss') + l2_reg*l2_beta
loss = tf.identity(loss, 'loss')

# Metric

In [0]:
# metric
logits_cls = tf.argmax(logits, axis=1)
logits_cls = tf.cast(logits_cls, dtype=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(ys, logits_cls),tf.float32), name='accuracy')

# Tensorboard 

In [0]:
tf.summary.scalar(name='accuracy', tensor=acc)
tf.summary.scalar(name='loss', tensor=loss)
merged = tf.summary.merge_all()

# Train OP

In [0]:
train_op = tf.train.MomentumOptimizer(lr, momentum=0.9).minimize(loss)

# Session Open 

In [0]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

init = tf.group(tf.global_variables_initializer(),
                tf.local_variables_initializer())
# saver 
saver = tf.train.Saver(max_to_keep=10)
sess.run(init)

# Tensorboard writer 

In [0]:
# tensorboard 
train_writer=tf.summary.FileWriter(logdir='./Resnet_1/log/train')
test_writer=tf.summary.FileWriter(logdir='./Resnet_1/log/test')

In [16]:
show_graph(graph_def=tf.get_default_graph())

In [16]:
dataprovider = DataProvider(images=x_train, labels=y_train)
save_root_folder = './Resnet_1/model'
os.makedirs(save_root_folder, exist_ok=True)

# hparam 
batch_size = 100
min_loss = 1000000.0


start_time = time.time()
for i in range(20000):
    learning_rate = 0.01
    batch_xs, batch_ys = dataprovider.next_batch(batch_size)
    # training 
    _ = sess.run(train_op, {xs: batch_xs, ys: batch_ys, lr: learning_rate, phase_train: True})

    if i % 100 == 0 :
        # Validate validation dataset 
        fetches=[loss, acc, merged]
        val_loss, val_acc, val_merged = sess.run(fetches,
                                                 {xs: x_test, ys: y_test, phase_train: False})

        # Validate train dataset : extract randomly 10000 samples from train dataset 
        batch_xs, batch_ys = dataprovider.next_batch(10000)
        train_loss, train_acc, train_merged = sess.run(fetches,
                                                       {xs: batch_xs, ys: batch_ys, phase_train: False})

        print('step : {} train loss : {:.4f} acc : {:.4f} | Val loss : {:.4f} acc : {:.4f}'.\
        format(i, train_loss, train_acc, val_loss, val_acc))

        # Save Model 
        if val_loss < min_loss:
            min_loss = val_loss
            save_path = os.path.join(save_root_folder, '{:.4f}_{:.4f}'.format(val_acc, val_loss))
            saver.save(sess=sess, save_path=save_path, global_step=i)
            print('model save!')

        # Add values to tensorboard 
        train_writer.add_summary(summary=train_merged, global_step=i)
        test_writer.add_summary(summary=val_merged, global_step=i)
        train_writer.flush()
print(
    time.time() - start_time  )

step : 0 train loss : 64.5476 acc : 0.0974 | Val loss : 64.3148 acc : 0.1000
model save!
step : 100 train loss : 1.5119 acc : 0.4384 | Val loss : 1.4983 acc : 0.4418
model save!
step : 200 train loss : 1.3324 acc : 0.5088 | Val loss : 1.3477 acc : 0.5013
model save!
step : 300 train loss : 1.1998 acc : 0.5624 | Val loss : 1.2108 acc : 0.5589
model save!
step : 400 train loss : 1.1384 acc : 0.5916 | Val loss : 1.1581 acc : 0.5858
model save!
step : 500 train loss : 1.0386 acc : 0.6276 | Val loss : 1.0649 acc : 0.6119
model save!
step : 600 train loss : 0.9766 acc : 0.6530 | Val loss : 1.0065 acc : 0.6382
model save!
step : 700 train loss : 0.9232 acc : 0.6679 | Val loss : 0.9738 acc : 0.6540
model save!
step : 800 train loss : 0.8262 acc : 0.7129 | Val loss : 0.8943 acc : 0.6858
model save!
step : 900 train loss : 0.8754 acc : 0.6945 | Val loss : 0.9409 acc : 0.6724
step : 1000 train loss : 0.7445 acc : 0.7396 | Val loss : 0.8043 acc : 0.7193
model save!
step : 1100 train loss : 0.7634 

W0705 11:18:15.733882 140305350174592 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


step : 1200 train loss : 0.7112 acc : 0.7534 | Val loss : 0.7979 acc : 0.7232
model save!
step : 1300 train loss : 0.6819 acc : 0.7654 | Val loss : 0.7932 acc : 0.7268
model save!
step : 1400 train loss : 0.6952 acc : 0.7574 | Val loss : 0.7932 acc : 0.7281
step : 1500 train loss : 0.6226 acc : 0.7819 | Val loss : 0.7202 acc : 0.7512
model save!
step : 1600 train loss : 0.6238 acc : 0.7780 | Val loss : 0.7109 acc : 0.7582
model save!
step : 1700 train loss : 0.6120 acc : 0.7860 | Val loss : 0.7390 acc : 0.7491
step : 1800 train loss : 0.5705 acc : 0.8025 | Val loss : 0.7185 acc : 0.7539
step : 1900 train loss : 0.5912 acc : 0.7964 | Val loss : 0.7293 acc : 0.7504
step : 2000 train loss : 0.5489 acc : 0.8096 | Val loss : 0.6873 acc : 0.7658
model save!
step : 2100 train loss : 0.5414 acc : 0.8099 | Val loss : 0.6688 acc : 0.7704
model save!
step : 2200 train loss : 0.5383 acc : 0.8114 | Val loss : 0.7102 acc : 0.7653
step : 2300 train loss : 0.5342 acc : 0.8131 | Val loss : 0.7187 acc :

KeyboardInterrupt: ignored